In [1]:
import gdal
import json
import geopandas as gp
from tqdm import tqdm
import os

#Shapefiles referentes aos chips de treinamento (treino, validação e teste)
chips_test = gp.GeoDataFrame.from_file(r'F:\MosaicosTrabalhados\chips_separation\training_sets\test_chips.shp')
chips_validation = gp.GeoDataFrame.from_file(r'F:\MosaicosTrabalhados\chips_separation\training_sets\validation_chips.shp')
chips_training = gp.GeoDataFrame.from_file(r'F:\MosaicosTrabalhados\chips_separation\training_sets\training_chips.shp')

#Rasters que serão utilizados para o treinamento da rede
ds_input = gdal.Open('F:/MosaicosBlob/mosaic_RGB_2.tif')
ds_target = gdal.Open('F:/MosaicosTrabalhados/chips_separation/rasters/estradas_buffered.tif')

#Retornar os chips em formato json para facilitar o acesso as informações do dataframe
def getFeatures(gdf):
    return json.loads(gdf.to_json())["features"]

training = getFeatures(chips_training)
validation = getFeatures(chips_validation)
test = getFeatures(chips_test)

#Degub para visualização das coordenadas de chips individuais
# xl = test[0]["geometry"]["coordinates"][0][0][0]
# yl = test[0]["geometry"]["coordinates"][0][0][1]
# xr = test[0]["geometry"]["coordinates"][0][2][0]
# yr = test[0]["geometry"]["coordinates"][0][2][1]

# print(xl)
# print(yl)
# print(xr)
# print(yr)

def chips_separation(chips_set, set_name):
    
    set_size = len(chips_set)
    
    for i in tqdm(range(0, set_size)):
        
        xl = chips_set[i]["geometry"]["coordinates"][0][0][0]
        yl = chips_set[i]["geometry"]["coordinates"][0][0][1]
        xr = chips_set[i]["geometry"]["coordinates"][0][2][0]
        yr = chips_set[i]["geometry"]["coordinates"][0][2][1]

        try:
#             gdal.Translate('F:/MosaicosTrabalhados/data/input/{}_buffered/{}_{}.tif'.format(set_name, set_name, str(i)), ds_input, projWin = [xl,yl,xr,yr])
            gdal.Translate('F:/MosaicosTrabalhados/data/target/{}_buffered/{}_{}.tif'.format(set_name, set_name, str(i)), ds_target, projWin = [xl,yl,xr,yr])
        except:
            print("Ocorreu um erro no processo de separação do chip.")
    

chips_separation(training, "training")
chips_separation(validation, "validation")
chips_separation(test, "test")

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:10<00:00, 24.33it/s]


In [3]:
from PIL import Image
import numpy as np
import numexpr as ne
import skimage.io as io
import tensorflow as tf
import gdal
import glob

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value.flatten()))

def write_image_annotation_pairs_to_tfrecord(filename_pairs, tfrecords_filename):
   
    writer = tf.io.TFRecordWriter(tfrecords_filename)
    
    zero_list = [np.zeros((1)) for x in range(0,255)]
    print("Começando conversão...")
    
    for img_path, annotation_path in list(filename_pairs):
        
        img = gdal.Open(img_path)
        img_b1 = np.array(img.GetRasterBand(1).ReadAsArray()).astype(np.float32())
        img_b2 = np.array(img.GetRasterBand(2).ReadAsArray()).astype(np.float32())
        img_b3 = np.array(img.GetRasterBand(3).ReadAsArray()).astype(np.float32())
        
        img_b1 = np.append(np.append(img_b1, zero_list, axis=1), np.zeros((1,256)), axis=0)
        img_b2 = np.append(np.append(img_b2, zero_list, axis=1), np.zeros((1,256)), axis=0)
        img_b3 = np.append(np.append(img_b3, zero_list, axis=1), np.zeros((1,256)), axis=0)
        
        def _linear_pct(arr, percent=2, minmax=(0, 255), ndv=None, dtype=np.float32,
                **kwargs):
           
            if isinstance(ndv, (int, float)):
                ndv = [ndv]

            if ndv:
                mask = ~np.in1d(arr, ndv).reshape(arr.shape)
                _min, _max = (np.percentile(arr[mask], percent),
                              np.percentile(arr[mask], 100 - percent))
            else:
                _min, _max = (np.percentile(arr, percent),
                              np.percentile(arr, 100 - percent))

            arr[arr >= _max] = _max
            arr[arr <= _min] = _min

            scale = (minmax[1] - minmax[0]) / (_max - _min)
            offset = minmax[1] - (scale * _max)

            return ne.evaluate('arr * scale + offset').astype(dtype)

        img_b1= _linear_pct(img_b1).astype(np.uint8())
        img_b2= _linear_pct(img_b2).astype(np.uint8())
        img_b3= _linear_pct(img_b3).astype(np.uint8())
        
        annotation = gdal.Open(annotation_path).ReadAsArray()
        annotation = np.array(annotation).astype(np.float32())
        annotation = np.append(np.append(annotation, zero_list, axis=1), np.zeros((1,256)), axis=0)

        example = tf.train.Example(features=tf.train.Features(feature={
            'b1': _float_feature(img_b1),
            'b2': _float_feature(img_b2),
            'b3': _float_feature(img_b3),
            'ref': _float_feature(annotation)}))

        writer.write(example.SerializeToString())

    writer.close()

training_paths = zip(glob.glob(r'F:\MosaicosTrabalhados\data\input\training\*.tif'), glob.glob(r'F:\MosaicosTrabalhados\data\target\training_buffered\*.tif'))
validation_paths = zip(glob.glob(r'F:\MosaicosTrabalhados\data\input\validation\*.tif'), glob.glob(r'F:\MosaicosTrabalhados\data\target\validation_buffered\*.tif'))
test_paths = zip(glob.glob(r'F:\MosaicosTrabalhados\data\input\test\*.tif'), glob.glob(r'F:\MosaicosTrabalhados\data\target\test_buffered\*.tif'))
    
write_image_annotation_pairs_to_tfrecord(training_paths, r'F:\MosaicosTrabalhados\data\training_buffered_0_255.tfrecord')
write_image_annotation_pairs_to_tfrecord(validation_paths, r'F:\MosaicosTrabalhados\data\validation_buffered_0_255.tfrecord')
write_image_annotation_pairs_to_tfrecord(test_paths, r'F:\MosaicosTrabalhados\data\test_buffered_0_255.tfrecord')

Começando conversão...
Começando conversão...
Começando conversão...


In [1]:
from azureml.tensorboard import Tensorboard

# The TensorBoard constructor takes an array of runs, so be sure and pass it in as a single-element array here
tb = Tensorboard([], local_root='./modelo_estradas_v1', port=6001)

# If successful, start() returns a string with the URI of the instance.
tb.start()

https://treino-estradas-6001.westeurope.instances.azureml.ms


'https://treino-estradas-6001.westeurope.instances.azureml.ms'

In [2]:
tb.stop()